In [4]:
import cobra
from cobra import io, flux_analysis
import os
#import pubchempy as pcp
#os.chdir(r"C:\Users\Bisbii\OneDrive - Universidade do Minho\Models")

In [5]:
model = cobra.io.read_sbml_model("yeast_lipidomics.xml")

In [6]:
def get_reaction_reactants_and_products(reaction):
    reactants = []
    products = []
    for metabolite in reaction.metabolites:
        if reaction.metabolites[metabolite] < 0:
            reactants.append(metabolite)
        else:
            products.append(metabolite)

    return reactants, products

In [7]:
sol = cobra.flux_analysis.pfba(model)
sol

,fluxes,reduced_costs
r_0001,0.000000,0.000000
r_0002,0.000000,-2.000000
r_0003,0.000000,-2.000000
r_0004,0.000000,-2.000000
r_0005,0.086385,-2.000000
...,...,...
r_4412,0.000000,52018.482587
r_4413,0.000000,89285.388998
r_4414,0.000000,114950.651367
r_4415,0.000000,178977.778852


In [8]:
slime_reactions = []
for reaction in model.reactions:
    if "SLIME" in reaction.name:
        slime_reactions.append(reaction.id)

In [9]:
for slime_reaction in slime_reactions:
    if sol.to_frame().loc[slime_reaction, "fluxes"] > 0:

        reaction = model.reactions.get_by_id(slime_reaction)
        reactants, products = get_reaction_reactants_and_products(reaction)

        print(reaction.name)
        print(sol[slime_reaction])
        print(reactants[0].formula_weight*sol[slime_reaction]/sol["r_2111"])

1-phosphatidyl-1D-myo-inositol (1-16:0, 2-16:1) [cytoplasm] SLIME rxn
3.3159135760928743e-22
2.877499816187132e-18
1-phosphatidyl-1D-myo-inositol (1-16:1, 2-16:1) [cytoplasm] SLIME rxn
0.0004201600061552645
3.6369999999999996
ergosteryl oleate [endoplasmic reticulum membrane] SLIME rxn
0.0005051345892298319
3.5820000000000003
stearate [cytoplasm] SLIME rxn
0.00025377901884294216
0.7716435212180718
oleate [cytoplasm] SLIME rxn
2.43673616056497e-06
0.007356479131422679
phosphatidyl-L-serine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.00039892675590779905
3.1319999999999997
phosphatidylcholine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.001127865476261059
8.831599307752676
phosphatidylcholine (1-16:0, 2-18:1) [endoplasmic reticulum membrane] SLIME rxn
7.060564510567146e-05
0.5756407024202206
phosphatidylethanolamine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.000494240364636765
3.647
triglyceride (1-16:0, 2-18:1, 3-16:0) [endoplasmic r

In [10]:
sol["r_4228"]

31.855378883769152

In [11]:
sol_fba = model.optimize()

In [12]:
for slime_reaction in slime_reactions:
    if sol_fba[slime_reaction] > 0:

        reaction = model.reactions.get_by_id(slime_reaction)
        reactants, products = get_reaction_reactants_and_products(reaction)

        print(reaction.name)
        print(sol_fba[slime_reaction])
        print(reactants[0].formula_weight*sol_fba[slime_reaction]/sol_fba["r_2111"])

ergosteryl oleate [endoplasmic reticulum membrane] SLIME rxn
0.0005051341918858265
3.582
stearate [cytoplasm] SLIME rxn
0.00025377881921064303
0.771643521196356
oleate [cytoplasm] SLIME rxn
2.436734135231305e-06
0.007356478803644112
phosphatidyl-L-serine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.00039892644210795574
3.132
phosphatidylcholine (1-16:0, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.0005301531492074693
4.162759987967962
phosphatidylcholine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.0011278645892237589
8.831599308951931
phosphatidylcholine (1-16:0, 2-18:1) [endoplasmic reticulum membrane] SLIME rxn
7.060558964749148e-05
0.575640703080105
phosphatidylethanolamine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.0004942399758622675
3.6469999999999994
triglyceride (1-16:0, 2-18:1, 3-16:0) [endoplasmic reticulum membrane] SLIME rxn
0.0004037385539625024
3.609
1-phosphatidyl-1D-myo-inositol (1-16:1, 2-16:1) [cytoplasm] S

In [13]:
fva = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=1, reaction_list=slime_reactions + ["r_2111"])

In [15]:
backbones_abundances = {}

for reaction in slime_reactions:
    if fva.loc[reaction, "maximum"] > 0:

        reaction = model.reactions.get_by_id(reaction)
        reactants, products = get_reaction_reactants_and_products(reaction)
        chains = {}
        backbone = ""
        for product in products:
            if "chain" in product.name:
                stoichiometry = reaction.get_coefficient(product)
                minimum = stoichiometry * fva.loc[reaction.id, "minimum"] / fva.loc["r_2111", "minimum"]
                maximum = stoichiometry * fva.loc[reaction.id, "maximum"] / fva.loc["r_2111", "maximum"]
                if product.name not in chains:
                    chains[product.name] = {"minimum": minimum, "maximum": maximum}
                else:
                    chains[product.name]["minimum"] += minimum
                    chains[product.name]["maximum"] += maximum
            else:
                backbone = product.name

        if backbone not in backbones_abundances:
            backbones_abundances[backbone] = chains
        else:
            for chain in chains:
                if chain not in backbones_abundances[backbone]:
                    backbones_abundances[backbone][chain] = chains[chain]
                else:
                    backbones_abundances[backbone][chain]["minimum"] += chains[chain]["minimum"]
                    backbones_abundances[backbone][chain]["maximum"] += chains[chain]["maximum"]


In [16]:
backbones_abundances

{'1-phosphatidyl-1D-myo-inositol [cytoplasm]': {'C16:0 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 1.6574295140481168},
  'C16:1 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 4.0287534534924685},
  'C18:0 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 0.08232326358848294},
  'C18:1 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 0.2269895957827579}},
 'ergosterol ester [cytoplasm]': {'C16:1 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 0.023233090865599836},
  'C18:1 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 3.060948769957319}},
 'fatty acid [cytoplasm]': {'C16:0 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 0.06157095796072368},
  'C16:1 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 0.048040286724160566},
  'C18:0 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 1.563542798588864},
  'C18:1 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 0.11215703711592062}},
 'phosphatidyl-L-serine [cytoplasm]': {'C16:0 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 1.

In [ ]:
def getPatwhaysMap(model):
    res = {}
    for pathway in model.groups:
        for reaction in pathway.members:
            if reaction.id not in res.keys():
                res[reaction.id] = pathway.name
            else:
                res[reaction.id] += "; " + pathway.name
    return res

def save_simulation(model, sol, filename, caption):
    solution = pd.DataFrame(sol.fluxes.copy())
    solution["Reaction Name"] = [None for el in solution.index]
    reaction_list = solution.index
    for reaction in reaction_list:
        reaction_name = model.reactions.get_by_id(reaction).name
        solution["Reaction Name"].loc[solution.index == reaction] = reaction_name
    solution = solution.loc[round(solution["fluxes"],3)!=0]
    solution = solution.sort_values(by="fluxes", axis=0)
    map_path = getPatwhaysMap(model)
    df = pd.DataFrame.from_dict(data = map_path, orient = 'index', columns = ["Pathways"])
    final = solution.merge(df, left_index=True, right_index=True, how="left")
    final.to_excel(filename + ".xlsx")
    df = model.summary(sol).to_frame()
    df = df.loc[df["flux"] != 0]
    df = df.drop(["factor"], axis=1)
    df = df.sort_values(by="flux", axis=0, ascending=False)
    df.columns = ["Reaction", "Metabolite", "Flux"]
    metabolites_list = df["Metabolite"].tolist()
    for met in metabolites_list:
        df["Metabolite"].loc[df["Metabolite"] == met] = model.metabolites.get_by_id(df["Metabolite"].loc[df["Metabolite"] == met].values[0]).name
    df_styled = df.style.hide_index().set_caption(caption).background_gradient()
    dfi.export(df_styled, filename + ".tiff")

In [ ]:
save_simulation(model, sol, "test", "a caption")

In [ ]:
fva_pfba = fva.merge(sol.to_frame(), left_index=True, right_index=True)
fva_pfba.to_csv("fva_solution.tsv", sep="\t")
fva_pfba